<a href="https://colab.research.google.com/github/JHyunjun/Data-Pre-Processing/blob/main/WavtoImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yt-dlp
!pip install pydub
!pip install librosa

import os
import yt_dlp
import librosa
from pydub import AudioSegment

# YouTube video URL
youtube_url = 'https://www.youtube.com/watch?v=I2ZEMjFJtzM'

# Download YouTube video as .wav audio file
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': 'downloaded_audio.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([youtube_url])

# If there is an error in running the youtube_dlp, please try to restart the runtime.

#%% Basic settings
audio_length = 4 # seconds
audio_length_ms = audio_length * 1000
data_overlap = 50 # percent
data_overlap_ps = data_overlap / 100
sampling_rate = 32768

os.makedirs("data_folder/wav_data", exist_ok=True)
os.makedirs("data_folder/mp3_data", exist_ok=True)

wav_path = "data_folder/wav_data"
mp3_path = "data_folder/mp3_data"

# Load the audio file
base_wav = AudioSegment.from_wav("downloaded_audio.wav")
audio = base_wav.set_frame_rate(sampling_rate)

# Segment the audio file and save each segment
num_segments = int(len(audio) / (audio_length_ms * data_overlap_ps))

for i in range(1, num_segments):
    tmp_fname_wav = f"audio_wav_{i:04}.wav"
    tmp_fname_mp3 = f"audio_mp3_{i:04}.mp3"
    tmp_audio = audio[(i-1)*audio_length_ms*data_overlap_ps : (i+1)*audio_length_ms*data_overlap_ps]
    tmp_audio.export(os.path.join(wav_path, tmp_fname_wav), format="wav")
    tmp_audio.export(os.path.join(mp3_path, tmp_fname_mp3), format="mp3")

# Load the segmented audio files and compute their STFT
n_fft = 1024
hop_length = 256

wav_files = os.listdir(wav_path)
mp3_files = os.listdir(mp3_path)

for i, file in enumerate(wav_files):
    y, sr = librosa.load(os.path.join(wav_path, file), sr=sampling_rate)
    S1 = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    globals()[f"wav_{i:04}"] = librosa.amplitude_to_db(S1, ref=1.0, amin=1e-05, top_db=80.0)

for i, file in enumerate(mp3_files):
    y, sr = librosa.load(os.path.join(mp3_path, file), sr=sampling_rate)
    S2 = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    globals()[f"mp3_{i:04}"] = librosa.amplitude_to_db(S2, ref=1.0, amin=1e-05, top_db=80.0)


In [ ]:
import matplotlib.pyplot as plt
import librosa.display

# Choose the first .wav and .mp3 file
wav_stft = globals()["wav_0000"]
mp3_stft = globals()["mp3_0000"]

print(wav_stft.shape)
print(mp3_stft.shape)

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 2)
librosa.display.specshow(wav_stft, sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram (.wav)')
plt.subplot(1, 2, 1)
librosa.display.specshow(mp3_stft, sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram (.mp3)')
plt.tight_layout()
plt.show()
